In [134]:
### TODO Normalize using groupby('file'); Add seconds column; Add column saying who won

In [135]:
# file for merging
# round 1-hot encoded
# tick and seconds normalize by match when figured out
# attck_team and vic_team depends on if predicting on dataset or outside dataset because uses only like 15 teams
# vic_side and att_side 1-hot encoded
# normalized hp_dmg and arm_dmg
# is_bomb_planted 1-hot encoded
# bomb_site filled nulls with 0 but we will 1-hot encode it
# need to 1-hot encode hitbox
# need to 1-hot encode 'wp' and 'wp type'
# att_id and vic_id is player id but once again will not really be used because it only works within dataset and not outside it
# att_rank and vic_rank only 1 value, which is 0
# att_pos_x, att_pos_y, vic_pos_x, vic_pos_y should not be normalized or one-hot encoded, so will not be included in X
# bomb site, hitbox, wp, wp type still need to be done

In [136]:
import pandas as pd
import numpy as np

In [137]:
copy_kills = pd.read_csv('csgo-matchmaking-damage/esea_master_kills_demos.part1.csv')

In [138]:
copy_meta = pd.read_csv('csgo-matchmaking-damage/esea_meta_demos.part1.csv')

In [139]:
master_kills = copy_kills
master_meta = copy_meta

In [140]:
master_kills = master_kills.drop(master_kills[(master_kills['ct_alive']<0) |(master_kills['t_alive'] < 0)].index).reset_index(drop=True)

In [141]:
master_kills.head()

,file,round,tick,seconds,att_team,vic_team,att_side,vic_side,wp,wp_type,ct_alive,t_alive,is_bomb_planted
0,esea_match_13770997.dem,1,16058,30.74165,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,USP,Pistol,5,4,False
1,esea_match_13770997.dem,1,16210,31.93185,Hentai Hooligans,Animal Style,Terrorist,CounterTerrorist,Glock,Pistol,4,4,False
2,esea_match_13770997.dem,1,16510,34.28094,Hentai Hooligans,Animal Style,Terrorist,CounterTerrorist,Glock,Pistol,3,4,False
3,esea_match_13770997.dem,1,17104,38.93212,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,USP,Pistol,3,3,False
4,esea_match_13770997.dem,1,17338,40.76441,Hentai Hooligans,Animal Style,Terrorist,CounterTerrorist,Glock,Pistol,2,3,False


In [142]:
master_kills.dtypes

file                object
round                int64
tick                 int64
seconds            float64
att_team            object
vic_team            object
att_side            object
vic_side            object
wp                  object
wp_type             object
ct_alive             int64
t_alive              int64
is_bomb_planted       bool
dtype: object

In [143]:
master_kills.isnull().apply(sum)

file                 0
round                0
tick                 0
seconds            135
att_team             0
vic_team             0
att_side             0
vic_side             0
wp                   0
wp_type              0
ct_alive             0
t_alive              0
is_bomb_planted      0
dtype: int64

In [144]:
master_kills = master_kills.dropna()

In [145]:
master_kills['att_side'].unique()#['att_team'].nunique()

array(['CounterTerrorist', 'Terrorist'], dtype=object)

In [146]:
master_kills['vic_side'].unique()#['att_team'].nunique()

array(['Terrorist', 'CounterTerrorist'], dtype=object)

In [147]:
att = pd.get_dummies(master_kills['att_side'])
att.head()

,CounterTerrorist,Terrorist
0,1,0
1,0,1
2,0,1
3,1,0
4,0,1


In [148]:
vic = pd.get_dummies(master_kills['vic_side'])
vic.head()

,CounterTerrorist,Terrorist
0,0,1
1,1,0
2,1,0
3,0,1
4,1,0


In [149]:
X = vic.join(att, lsuffix='_vic', rsuffix='_att')

In [150]:
master_kills['file'].unique()

array(['esea_match_13770997.dem', 'esea_match_13779704.dem',
       'esea_match_13779769.dem', ..., 'esea_match_13793001.dem',
       'esea_match_13793017.dem', 'esea_match_13793043.dem'], dtype=object)

In [151]:
master_kills.columns

Index(['file', 'round', 'tick', 'seconds', 'att_team', 'vic_team', 'att_side',
       'vic_side', 'wp', 'wp_type', 'ct_alive', 't_alive', 'is_bomb_planted'],
      dtype='object')

In [152]:
round_num = pd.get_dummies(master_kills['round'])

In [153]:
round_num.columns = ['round_' +str(i) for i in round_num.columns]
round_num.head()

,round_1,round_2,round_3,round_4,round_5,round_6,round_7,round_8,round_9,round_10,...,round_54,round_55,round_56,round_57,round_58,round_59,round_60,round_61,round_62,round_63
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [154]:
X = X.join(round_num)

In [155]:
X = X.rename(columns = {'None':'None_att'})

In [156]:
X.head()

,CounterTerrorist_vic,Terrorist_vic,CounterTerrorist_att,Terrorist_att,round_1,round_2,round_3,round_4,round_5,round_6,...,round_54,round_55,round_56,round_57,round_58,round_59,round_60,round_61,round_62,round_63
0,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [157]:
master_kills.columns

Index(['file', 'round', 'tick', 'seconds', 'att_team', 'vic_team', 'att_side',
       'vic_side', 'wp', 'wp_type', 'ct_alive', 't_alive', 'is_bomb_planted'],
      dtype='object')

In [158]:
bomb_planted = pd.get_dummies(master_kills['is_bomb_planted'])
bomb_planted.columns = ['bomb_planted_False', 'bomb_planted_True']

In [159]:
X = X.join(bomb_planted)

In [160]:
master_kills['wp'].unique()

array(['USP', 'Glock', 'AK47', 'Deagle', 'CZ', 'Negev', 'Mac10', 'M4A4',
       'P250', 'Bizon', 'AWP', 'P90', 'Famas', 'Swag7', 'MP9', 'UMP',
       'M4A1', 'FiveSeven', 'Gallil', 'XM1014', 'HE', 'Scout', 'Nova',
       'MP7', 'Scar20', 'Incendiary', 'AUG', 'P2000', 'Knife', 'SG556',
       'Tec9', 'SawedOff', 'DualBarettas', 'Zeus', 'G3SG1', 'Revolver',
       'World', 'M249', 'Flash', 'Molotov', 'Smoke'], dtype=object)

In [161]:
X = X.join(pd.get_dummies(master_kills['wp']))

In [162]:
X = X.join(pd.get_dummies(master_kills['wp_type']).rename(columns={'Unkown':'Unknown_type'}))

In [163]:
master_kills[master_kills['wp_type']=='Unkown']

,file,round,tick,seconds,att_team,vic_team,att_side,vic_side,wp,wp_type,ct_alive,t_alive,is_bomb_planted
43805,esea_match_13780167.dem,1,6475,50.62688,Team 2,Team 2,Terrorist,Terrorist,World,Unkown,5,4,False
278723,esea_match_13782173.dem,3,45284,19.47864,Team 1,Team 1,Terrorist,CounterTerrorist,World,Unkown,5,4,False
329312,esea_match_13782584.dem,9,113078,15.02686,Team 1,Team 1,Terrorist,CounterTerrorist,World,Unkown,5,4,False
698280,esea_match_13785667.dem,2,24473,15.00482,Team 2,Team 2,CounterTerrorist,Terrorist,World,Unkown,4,5,False
906468,esea_match_13787373.dem,12,144946,15.16772,Team 1,Team 1,Terrorist,CounterTerrorist,World,Unkown,5,4,False
1332060,esea_match_13791050.dem,25,248269,22.55115,Team 2,Team 2,Terrorist,CounterTerrorist,World,Unkown,5,4,False


In [164]:
master_kills['ct_alive'].unique()

array([5, 4, 3, 2, 1, 0])

In [165]:
ct_alive = pd.get_dummies(master_kills['ct_alive'])
t_alive = pd.get_dummies(master_kills['t_alive'])

In [166]:
ct_alive.columns = [str(i)+'_ct' for i in ct_alive.columns]

In [167]:
t_alive.columns = [str(i)+'_t' for i in t_alive.columns]

In [168]:
ct_alive.head()

,0_ct,1_ct,2_ct,3_ct,4_ct,5_ct
0,0,0,0,0,0,1
1,0,0,0,0,1,0
2,0,0,0,1,0,0
3,0,0,0,1,0,0
4,0,0,1,0,0,0


In [169]:
X = X.join(ct_alive)
X = X.join(t_alive)

In [170]:
X = X.reset_index(drop=True)

In [171]:
master_kills.head()

,file,round,tick,seconds,att_team,vic_team,att_side,vic_side,wp,wp_type,ct_alive,t_alive,is_bomb_planted
0,esea_match_13770997.dem,1,16058,30.74165,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,USP,Pistol,5,4,False
1,esea_match_13770997.dem,1,16210,31.93185,Hentai Hooligans,Animal Style,Terrorist,CounterTerrorist,Glock,Pistol,4,4,False
2,esea_match_13770997.dem,1,16510,34.28094,Hentai Hooligans,Animal Style,Terrorist,CounterTerrorist,Glock,Pistol,3,4,False
3,esea_match_13770997.dem,1,17104,38.93212,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,USP,Pistol,3,3,False
4,esea_match_13770997.dem,1,17338,40.76441,Hentai Hooligans,Animal Style,Terrorist,CounterTerrorist,Glock,Pistol,2,3,False


Now we will join master_meta to master_kills based on 'file' and 'round'

In [172]:
master_meta.head()

,file,map,round,start_seconds,end_seconds,winner_team,winner_side,round_type,ct_eq_val,t_eq_val
0,esea_match_13770997.dem,de_overpass,1,94.30782,160.9591,Hentai Hooligans,Terrorist,PISTOL_ROUND,4300,4250
1,esea_match_13770997.dem,de_overpass,2,160.95910,279.3998,Hentai Hooligans,Terrorist,ECO,6300,19400
2,esea_match_13770997.dem,de_overpass,3,279.39980,341.0084,Hentai Hooligans,Terrorist,SEMI_ECO,7650,19250
3,esea_match_13770997.dem,de_overpass,4,341.00840,435.4259,Hentai Hooligans,Terrorist,NORMAL,24900,23400
4,esea_match_13770997.dem,de_overpass,5,435.42590,484.2398,Animal Style,CounterTerrorist,ECO,5400,20550


In [173]:
master_kills.head()

,file,round,tick,seconds,att_team,vic_team,att_side,vic_side,wp,wp_type,ct_alive,t_alive,is_bomb_planted
0,esea_match_13770997.dem,1,16058,30.74165,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,USP,Pistol,5,4,False
1,esea_match_13770997.dem,1,16210,31.93185,Hentai Hooligans,Animal Style,Terrorist,CounterTerrorist,Glock,Pistol,4,4,False
2,esea_match_13770997.dem,1,16510,34.28094,Hentai Hooligans,Animal Style,Terrorist,CounterTerrorist,Glock,Pistol,3,4,False
3,esea_match_13770997.dem,1,17104,38.93212,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,USP,Pistol,3,3,False
4,esea_match_13770997.dem,1,17338,40.76441,Hentai Hooligans,Animal Style,Terrorist,CounterTerrorist,Glock,Pistol,2,3,False


In [174]:
master_join = master_kills.merge(master_meta, on=['file', 'round'], how='outer')

In [175]:
len(master_join)

1569085

In [176]:
len(master_join[master_join['start_seconds'].isnull()]) / len(master_join)

0.017508293049771046

In [177]:
master_join.isnull().sum()

file                   0
round                  0
tick                  20
seconds               20
att_team              20
vic_team              20
att_side              20
vic_side              20
wp                    20
wp_type               20
ct_alive              20
t_alive               20
is_bomb_planted       20
map                27452
start_seconds      27472
end_seconds        27472
winner_team        27459
winner_side        27452
round_type         27452
ct_eq_val          27452
t_eq_val           27452
dtype: int64

In [178]:
master_join = master_join[master_join['tick'].isnull()==False]

In [179]:
master_join.isnull().sum()

file                   0
round                  0
tick                   0
seconds                0
att_team               0
vic_team               0
att_side               0
vic_side               0
wp                     0
wp_type                0
ct_alive               0
t_alive                0
is_bomb_planted        0
map                27452
start_seconds      27452
end_seconds        27452
winner_team        27459
winner_side        27452
round_type         27452
ct_eq_val          27452
t_eq_val           27452
dtype: int64

In [180]:
len(master_join)

1569065

In [181]:
len(X) 

1569065

In [182]:
master_join = master_join.reset_index(drop=True)
X = X.reset_index(drop=True)

In [183]:
X = X[master_join['winner_team'].isnull() == False]


In [195]:
len(X)

1541606

In [185]:
X.head()

,CounterTerrorist_vic,Terrorist_vic,CounterTerrorist_att,Terrorist_att,round_1,round_2,round_3,round_4,round_5,round_6,...,2_ct,3_ct,4_ct,5_ct,0_t,1_t,2_t,3_t,4_t,5_t
0,0,1,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,1,0,0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,1,0,0,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,0,1,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,1,0,0,1,1,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0


In [186]:
len(master_join)

1569065

In [187]:
len(X)

1541606

In [118]:
len(X) == len(master_join)

False

In [188]:
master_join = master_join.dropna()

In [189]:
len(master_join)

1541606

In [190]:
len(X) == len(master_join)

True

In [191]:
y = pd.get_dummies(master_join['winner_side'])

In [192]:
len(X) == len(y)

True

In [196]:
len(X)

1541606

In [197]:
master_join.head()

,file,round,tick,seconds,att_team,vic_team,att_side,vic_side,wp,wp_type,...,t_alive,is_bomb_planted,map,start_seconds,end_seconds,winner_team,winner_side,round_type,ct_eq_val,t_eq_val
0,esea_match_13770997.dem,1,16058.0,30.74165,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,USP,Pistol,...,4.0,False,de_overpass,94.30782,160.9591,Hentai Hooligans,Terrorist,PISTOL_ROUND,4300.0,4250.0
1,esea_match_13770997.dem,1,16210.0,31.93185,Hentai Hooligans,Animal Style,Terrorist,CounterTerrorist,Glock,Pistol,...,4.0,False,de_overpass,94.30782,160.9591,Hentai Hooligans,Terrorist,PISTOL_ROUND,4300.0,4250.0
2,esea_match_13770997.dem,1,16510.0,34.28094,Hentai Hooligans,Animal Style,Terrorist,CounterTerrorist,Glock,Pistol,...,4.0,False,de_overpass,94.30782,160.9591,Hentai Hooligans,Terrorist,PISTOL_ROUND,4300.0,4250.0
3,esea_match_13770997.dem,1,17104.0,38.93212,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,USP,Pistol,...,3.0,False,de_overpass,94.30782,160.9591,Hentai Hooligans,Terrorist,PISTOL_ROUND,4300.0,4250.0
4,esea_match_13770997.dem,1,17338.0,40.76441,Hentai Hooligans,Animal Style,Terrorist,CounterTerrorist,Glock,Pistol,...,3.0,False,de_overpass,94.30782,160.9591,Hentai Hooligans,Terrorist,PISTOL_ROUND,4300.0,4250.0


Now we have our features and labels for esea_master_kills_demos.part1.csv

In [199]:
X['seconds_norm'] = master_join.groupby('file')['seconds'].transform(lambda x: (x - x.mean()) / x.std())

In [200]:
maps = pd.get_dummies(master_join['map'])
maps.head()

,de_cache,de_cbble,de_dust2,de_inferno,de_mirage,de_nuke,de_overpass,de_train
0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,1,0


In [201]:
X = X.join(maps)

In [202]:
round_types = pd.get_dummies(master_join['round_type'])
round_types.head()

,ECO,FORCE_BUY,NORMAL,PISTOL_ROUND,SEMI_ECO
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,1,0
3,0,0,0,1,0
4,0,0,0,1,0


In [203]:
X = X.join(round_types)

In [204]:
X['ct_eq_val_norm'] = master_join.groupby('file')['ct_eq_val'].transform(lambda x: (x - x.mean()) / x.std())
X['t_eq_val_norm'] = master_join.groupby('file')['t_eq_val'].transform(lambda x: (x - x.mean()) / x.std())

In [206]:
X = X.reset_index(drop=True)

In [207]:
X.to_csv('CSGO_Data_Cleaning/esea_master_kills_meta_cleaned.part1.csv')

In [208]:
X

,CounterTerrorist_vic,Terrorist_vic,CounterTerrorist_att,Terrorist_att,round_1,round_2,round_3,round_4,round_5,round_6,...,de_nuke,de_overpass,de_train,ECO,FORCE_BUY,NORMAL,PISTOL_ROUND,SEMI_ECO,ct_eq_val_norm,t_eq_val_norm
0,0,1,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,-1.676921,-1.625270
1,1,0,0,1,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,-1.676921,-1.625270
2,1,0,0,1,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,-1.676921,-1.625270
3,0,1,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,-1.676921,-1.625270
4,1,0,0,1,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,-1.676921,-1.625270
5,1,0,0,1,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,-1.676921,-1.625270
6,0,1,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,-1.676921,-1.625270
7,0,1,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,-1.676921,-1.625270
8,1,0,0,1,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,-1.676921,-1.625270
9,1,0,0,1,0,1,0,0,0,0,...,0,1,0,1,0,0,0,0,-1.489767,0.194592
